## 모듈

In [363]:
import pandas as pd
import numpy as np

In [364]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Data

In [403]:
Distance=pd.read_csv("/content/drive/MyDrive/Best Key_2022산업공학종합설계/거리matrix/Distance_matrix_행운동.csv")
needs=pd.read_excel("/content/drive/MyDrive/Best Key_2022산업공학종합설계/거리matrix/좌표_최종_행운동.xlsx")

In [404]:
del Distance['Unnamed: 0']

In [405]:
# 문자열 -> km 단위의 실수형으로 변환

for j in range(Distance.shape[1]):
    for i in range(Distance.shape[0]):
        Distance.iloc[i, j] = Distance.iloc[i, j].replace(' km', '')
    Distance.iloc[:, j] = Distance.iloc[:, j].astype(float)
Distance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161 entries, 0 to 160
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   행운동 주민센터           161 non-null    float64
 1   행운동제1공영주차장         161 non-null    float64
 2   까치어린이공원            161 non-null    float64
 3   장미어린이공원            161 non-null    float64
 4   행운치안센터             161 non-null    float64
 5   행운동 경로당 (구립행운경로당)  161 non-null    float64
 6   원당경로당              161 non-null    float64
 7   서울대입구역             161 non-null    float64
dtypes: float64(8)
memory usage: 10.2 KB


## P-Median

In [9]:
pip install pulp

     |████████████████████████████████| 14.2 MB 4.7 MB/s 


In [208]:
from pulp import *

In [383]:
I = [i for i in range(location.shape[0])]

In [385]:
location=I

In [393]:
blocks=[]
for i in range(0,161):
  blocks.append(i)

In [395]:
a=dict(zip(blocks,demand))

In [398]:
D={}
for i in range(0, 161):
  D[i]=dict(zip(blocks[i:],[dict(zip(location,list(Distance.iloc[i])))]))

In [399]:
merged={}
for i in range(0,161):
  merged={**merged,**D[i]}

In [400]:
merged

{0: {0: 1.4, 1: 1.4, 2: 1.4, 3: 1.4, 4: 1.4, 5: 1.4, 6: 1.4, 7: 1.4},
 1: {0: 1.4, 1: 1.4, 2: 1.4, 3: 1.4, 4: 1.4, 5: 1.4, 6: 1.4, 7: 1.4},
 2: {0: 1.3, 1: 1.3, 2: 1.3, 3: 1.3, 4: 1.3, 5: 1.3, 6: 1.3, 7: 1.3},
 3: {0: 0.5, 1: 0.5, 2: 0.5, 3: 0.5, 4: 0.5, 5: 0.5, 6: 0.5, 7: 0.5},
 4: {0: 0.7, 1: 0.7, 2: 0.7, 3: 0.7, 4: 0.7, 5: 0.7, 6: 0.7, 7: 0.7},
 5: {0: 1.6, 1: 1.6, 2: 1.6, 3: 1.6, 4: 1.6, 5: 1.6, 6: 1.6, 7: 1.6},
 6: {0: 1.6, 1: 1.6, 2: 1.6, 3: 1.6, 4: 1.6, 5: 1.6, 6: 1.6, 7: 1.6},
 7: {0: 1.6, 1: 1.6, 2: 1.6, 3: 1.6, 4: 1.6, 5: 1.6, 6: 1.6, 7: 1.6},
 8: {0: 1.5, 1: 1.5, 2: 1.5, 3: 1.5, 4: 1.5, 5: 1.5, 6: 1.5, 7: 1.5},
 9: {0: 1.5, 1: 1.5, 2: 1.5, 3: 1.5, 4: 1.5, 5: 1.5, 6: 1.5, 7: 1.5},
 10: {0: 0.6, 1: 0.6, 2: 0.6, 3: 0.6, 4: 0.6, 5: 0.6, 6: 0.6, 7: 0.6},
 11: {0: 1.7, 1: 1.7, 2: 1.7, 3: 1.7, 4: 1.7, 5: 1.7, 6: 1.7, 7: 1.7},
 12: {0: 0.6, 1: 0.6, 2: 0.6, 3: 0.6, 4: 0.6, 5: 0.6, 6: 0.6, 7: 0.6},
 13: {0: 0.6, 1: 0.6, 2: 0.6, 3: 0.6, 4: 0.6, 5: 0.6, 6: 0.6, 7: 0.6},
 14: {0: 1.7, 1:

In [401]:
#P-median
X=LpVariable.dicts('X_%s_%s',(blocks,location),cat='Binary',lowBound=0,upBound=1)
p=1

prob =LpProblem('P Median',LpMinimize)#목적함수
prob +=sum(sum(a[i]*merged[i][j]*X[i][j] for j in location) for i in blocks)
prob +=sum(X[j][j] for j in location) == p
             
#제약식
for i in blocks : prob += sum(X[i][j] for j in location ) == 1
for i in blocks:
  for j in location : prob +=X[i][j] <=X[j][j]

/usr/local/lib/python3.7/dist-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [402]:
prob.solve()
print("Status:" ,LpStatus[prob.status])
print("Objective:",value(prob.objective))
for v in prob.variables(): 
  print (v.name , "=", v.varValue)

Status: Optimal
Objective: 843414450.5000004
X_0_0 = 0.0
X_0_1 = 0.0
X_0_2 = 0.0
X_0_3 = 0.0
X_0_4 = 0.0
X_0_5 = 0.0
X_0_6 = 0.0
X_0_7 = 1.0
X_100_0 = 0.0
X_100_1 = 0.0
X_100_2 = 0.0
X_100_3 = 0.0
X_100_4 = 0.0
X_100_5 = 0.0
X_100_6 = 0.0
X_100_7 = 1.0
X_101_0 = 0.0
X_101_1 = 0.0
X_101_2 = 0.0
X_101_3 = 0.0
X_101_4 = 0.0
X_101_5 = 0.0
X_101_6 = 0.0
X_101_7 = 1.0
X_102_0 = 0.0
X_102_1 = 0.0
X_102_2 = 0.0
X_102_3 = 0.0
X_102_4 = 0.0
X_102_5 = 0.0
X_102_6 = 0.0
X_102_7 = 1.0
X_103_0 = 0.0
X_103_1 = 0.0
X_103_2 = 0.0
X_103_3 = 0.0
X_103_4 = 0.0
X_103_5 = 0.0
X_103_6 = 0.0
X_103_7 = 1.0
X_104_0 = 0.0
X_104_1 = 0.0
X_104_2 = 0.0
X_104_3 = 0.0
X_104_4 = 0.0
X_104_5 = 0.0
X_104_6 = 0.0
X_104_7 = 1.0
X_105_0 = 0.0
X_105_1 = 0.0
X_105_2 = 0.0
X_105_3 = 0.0
X_105_4 = 0.0
X_105_5 = 0.0
X_105_6 = 0.0
X_105_7 = 1.0
X_106_0 = 0.0
X_106_1 = 0.0
X_106_2 = 0.0
X_106_3 = 0.0
X_106_4 = 0.0
X_106_5 = 0.0
X_106_6 = 0.0
X_106_7 = 1.0
X_107_0 = 0.0
X_107_1 = 0.0
X_107_2 = 0.0
X_107_3 = 0.0
X_107_4 = 0.0
X_107